In [1]:
import numpy as np
from cellpose import utils, io
from cellpose import models
import skimage.io
from cellpose import plot
import os
import cv2
from image_slicer import slice
from image_slicer import join

import pandas as pd

from scipy import stats

from glob import glob

from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
import re

C:\Users\14194\anaconda3\envs\cellpose\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
#first we will create a composite image of all the combination of images in cellpose 
membrane_list=[]
nuclei_list=[]
input_dir=r'C:\Users\14194\Documents\08222023_DR1116_0k\Images'




for r, d, f in os.walk(input_dir):
    for fil in f:
        if fil.endswith('mem.tif'): #whatever string signifies the end sequence of your membrane file, in this case"mem"
            membrane_list.append(os.path.join(input_dir, fil))
        elif fil.endswith('nuc.tif'):
            nuclei_list.append(os.path.join(input_dir, fil)) #nuclei is whatever files dont make first criteria
#now we will create a composite image for better cell predictions



In [3]:
nuclei_list.sort()
membrane_list.sort()


27
27


In [4]:
composite_image_folder=r'C:\Users\14194\Documents\08222023_DR1116_0k\Composites' # save composite images, channels are [membrane, nuclei]
# create composite images for each of what we care about
for i in range(len(membrane_list)):
    Membrane=io.imread(membrane_list[i])
    Nuclei=io.imread(nuclei_list[i])

    composite=np.stack((Membrane,Nuclei))
    composite_name=os.path.basename(membrane_list[i])[:-7]+'_composite.tif'
    composite_output=os.path.join(composite_image_folder, composite_name)
    skimage.io.imsave(composite_output,composite)
#save composite images 
    
    
    
    # now we will create a composite list with the path of all the images we care about

Composite_List=[]
for r, d, f in os.walk(composite_image_folder):
    for fil in f:
        if fil.endswith('.tif'):
            Composite_List.append(os.path.join(composite_image_folder, fil))

C:\Users\14194\AppData\Local\Temp\ipykernel_23904\332382765.py:10: UserWarning: C:\Users\14194\Documents\08222023_DR1116_0k\Composites\08222023_DR1116_0k_A_2__composite.tif is a low contrast image
  skimage.io.imsave(composite_output,composite)
C:\Users\14194\AppData\Local\Temp\ipykernel_23904\332382765.py:10: UserWarning: C:\Users\14194\Documents\08222023_DR1116_0k\Composites\08222023_DR1116_0k_B_1__composite.tif is a low contrast image
  skimage.io.imsave(composite_output,composite)
C:\Users\14194\AppData\Local\Temp\ipykernel_23904\332382765.py:10: UserWarning: C:\Users\14194\Documents\08222023_DR1116_0k\Composites\08222023_DR1116_0k_B_2__composite.tif is a low contrast image
  skimage.io.imsave(composite_output,composite)
C:\Users\14194\AppData\Local\Temp\ipykernel_23904\332382765.py:10: UserWarning: C:\Users\14194\Documents\08222023_DR1116_0k\Composites\08222023_DR1116_0k_B_3__composite.tif is a low contrast image
  skimage.io.imsave(composite_output,composite)
C:\Users\14194\AppDa

# Now we have the composite images, the following will iterate through the images, infer both a membrane and  nuclei mask, and save them in the corresponding folder. These are initial conditions that will be processed in the subsequent code.

In [7]:
nuclei_mask_path=r'C:\Users\14194\Documents\08222023_DR1116_0k\Nuclei Masks' #where to save nuclei masks
cell_mask_path=r'C:\Users\14194\Documents\08222023_DR1116_0k\Cell Masks' #where to save cell (membrane) masks

channels_cell=[1,2] #cell is the R channel, Nuclei is the G channel for cell segmentation
channels_nuclei=[2,0] #for nuclei segmentation

for i in range(len(Composite_List)):
    model_nuclei=models.Cellpose(gpu=True,model_type='nuclei')
    model_cell=models.Cellpose(gpu=True,model_type='cyto')
    img=io.imread(Composite_List[i])

    masks, flows, styles, diams = model_nuclei.eval(img,  channels=channels_nuclei,  diameter=9,flow_threshold=0.4,do_3D=False,min_size=15, batch_size = 16)
    nuclei_name=os.path.basename(membrane_list[i])[:-7]+'_nuclei_mask.tif'
    nuclei_output=os.path.join(nuclei_mask_path, nuclei_name)
    mask=masks.astype(np.uint16)
    skimage.io.imsave(nuclei_output,mask, check_contrast=False)
        #nuc_masks.append(masks)
    masks, flows, styles, diams = model_cell.eval(img,  channels=channels_cell, diameter=12,cellprob_threshold=0.0,flow_threshold=0.4, min_size=15, do_3D=False, batch_size = 16)
    cell_name=os.path.basename(membrane_list[i])[:-7]+'_cell_mask.tif'
    cell_output=os.path.join(cell_mask_path, cell_name)
    mask=masks.astype(np.uint16)
    skimage.io.imsave(cell_output,mask, check_contrast=False)

# Read in all the cell and Nuclei mask paths 

In [8]:
Membrane_list=[]
Nuclei_list=[]
input_dir=r'C:\Users\14194\Documents\08222023_DR1116_0k\Cell Masks'


for r, d, f in os.walk(input_dir): #appending to actin list
    for fil in f:
        if fil.endswith('mask.tif'):
            Membrane_list.append(os.path.join(input_dir, fil))
        

input_dir=r'C:\Users\14194\Documents\08222023_DR1116_0k\Nuclei Masks'


for r, d, f in os.walk(input_dir): #appending to nuclei list 
    for fil in f:
        if fil.endswith('mask.tif'):
            Nuclei_list.append(os.path.join(input_dir, fil))

# The following will "connect" the cell and nuclei masks

1) Ensure each nuclei and cell are matched 1:1 with same pixel value

2) ensure no 2 nuclei are within a cell

3) ensure nuclei are not bigger than the cell

4) save the connected masks in "save_path_cell"

In [9]:
save_path_cell=r'C:\Users\14194\Documents\08222023_DR1116_0k\connected masks'
save_path_nuclei=save_path_cell
#savepaths


for i in range(len(Membrane_list)): #iterating through the image list
    cell_image=cv2.imread(Membrane_list[i],-1);cell_name=os.path.basename(Membrane_list[i])
    nuclei_image=cv2.imread(Nuclei_list[i],-1);Nuclei_name=os.path.basename(Nuclei_list[i])
    pixel_list=np.unique(cell_image)[1:] # to drop pixel value of 0 
    for index in pixel_list:
        pixel_cell_reduced=cell_image.copy()
        pixel_cell_reduced[cell_image!=index]=[0]
        pixel_nuclei_reduced=nuclei_image.copy()
        pixel_nuclei_reduced[cell_image!=index]=[0]
        nuclei_list=np.unique(pixel_nuclei_reduced)
        if len(nuclei_list)>=2:
            nuclei_index=nuclei_list[-1]
            nuclei_image[pixel_nuclei_reduced==nuclei_index]=[index]
        else:
            cell_image[cell_image==index]=[0]
            for k in nuclei_list:
                nuclei_image[nuclei_image==k]=[0]
    #now we clear borders
    cell_image=clear_border(cell_image)
    
    #stuff that is in nuclei image not in cell image are being dropped
    droplist=np.setdiff1d(nuclei_image,cell_image)
    for item in droplist:
        nuclei_image[nuclei_image==item]=[0]
    droplist=np.setdiff1d(cell_image,nuclei_image)
    for item in droplist:
        cell_image[cell_image==item]=[0]

    cv2.imwrite(os.path.join(save_path_cell,cell_name),cell_image) #save cell mask
    cv2.imwrite(os.path.join(save_path_nuclei,Nuclei_name),nuclei_image)# save nuclei mask 

# Pandas Dataframe constructor

1) here we will create a dataframe with some basic parameters (this portion of the code can definitely nbe optimized)

In [10]:
Membrane_list=[]
Nuclei_list=[]
input_dir=r'C:\Users\14194\Documents\08222023_DR1116_0k\connected masks'


for r, d, f in os.walk(input_dir): #appending to actin list
    for fil in f:
        if fil.endswith('cell_mask.tif'):
            Membrane_list.append(os.path.join(input_dir, fil))
        



for r, d, f in os.walk(input_dir): #appending to nuclei list 
    for fil in f:
        if fil.endswith('nuclei_mask.tif'):
            Nuclei_list.append(os.path.join(input_dir, fil))

The folllowing will create a list for each parameter we want to identify

In [11]:
Nuclei_File=[]
Membrane_File=[]
Pixel_Coordinate=[]
Nuclear_Area=[]
Cell_Area=[]
N_C_Ratio=[]
circularity = [] 
eccentricity = []

for i in range(len(Membrane_list)):
    nuclei=cv2.imread(Nuclei_list[i],-1)
    membrane=cv2.imread(Membrane_list[i],-1)
    for j in np.unique(nuclei)[1:]:        
        regions = regionprops((nuclei == j).astype(int))
        circularity.append((4*np.pi*(regions[0].area)/(regions[0].perimeter_crofton)**2))
        eccentricity.append(regions[0].eccentricity)
        Nuclei_File.append(os.path.basename(Nuclei_list[i])) #nuclei name of mask
        Membrane_File.append(os.path.basename(Membrane_list[i]))#cell name of mask
        Pixel_Coordinate.append(j) #pixel coordinate of a cell
        Nuclear_Area.append(np.count_nonzero(nuclei == j)) # area of nuclei (how many of that pixel value are in the image array)
        Cell_Area.append(np.count_nonzero(membrane == j)) #same as above but for cell
        N_C_Ratio.append(np.count_nonzero(nuclei == j)/np.count_nonzero(membrane == j)) # dividing nuclei by cell area

In [12]:
Master_Sheet=pd.DataFrame() # creating dataframe
Master_Sheet['Nuclei_File']=Nuclei_File
Master_Sheet['Membrane_File']=Membrane_File
Master_Sheet['Pixel_Coordinate']=Pixel_Coordinate
Master_Sheet['Nuclear_Area (pixels)']=Nuclear_Area
Master_Sheet['Cell_Area (pixels)']=Cell_Area
Master_Sheet['N:C_Ratio']=N_C_Ratio
Master_Sheet['circ.']= circularity
Master_Sheet["Eccentricity"] = eccentricity

In [13]:
Master_Sheet["Bio rep"]= Master_Sheet["Nuclei_File"].apply(lambda x: str(x.split("_")[:4]))

In [14]:
Master_Sheet["TechRep"]= Master_Sheet["Nuclei_File"].apply(lambda x: str(x.split("_")[:5]))

In [15]:
Master_Sheet["strain"]= Master_Sheet["Nuclei_File"].apply(lambda x: str(x.split("_")[:3]))

In [17]:
result = Master_Sheet.groupby(['TechRep']).agg(['mean','std','count'])

C:\Users\14194\AppData\Local\Temp\ipykernel_23904\4205953086.py:1: FutureWarning: ['Nuclei_File', 'Membrane_File', 'Bio rep', 'strain'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  result = Master_Sheet.groupby(['TechRep']).agg(['mean','std','count'])


In [19]:
Master_Sheet.to_csv(r"\Users\14194\Documents\08222023_DR1116_0k\08222023_DR1116_0k_master_sheet.csv")

In [20]:
result.to_csv(r"\Users\14194\Documents\08222023_DR1116_0k\08222023_DR1116_0k_grouped_results.csv")